In [11]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.sparse import hstack, csr_matrix
import numpy as np
import streamlit as st
import pickle
import os
import pdfplumber
from docx import Document
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
# nltk.download("stopwords")

In [12]:
# import sys
# print(sys.executable)

In [13]:
# !"C:\Users\wajih\anaconda3\envs\all_ai_env\python.exe" -m pip install streamlit pdfplumber python-docx nltk scikit-learn scipy pandas numpy

In [14]:
# =====================
# Load Data
# =====================
df = pd.read_csv("resume_data.csv")

In [15]:
print("Shape:", df.shape)

Shape: (9544, 35)


In [16]:
df.head()

,address,career_objective,skills,educational_institution_name,degree_names,passing_years,educational_results,result_types,major_field_of_studies,professional_company_names,...,online_links,issue_dates,expiry_dates,﻿job_position_name,educationaL_requirements,experiencere_requirement,age_requirement,responsibilities.1,skills_required,matched_score
0,NaN,Big data analytics working and database wareho...,"['Big Data', 'Hadoop', 'Hive', 'Python', 'Mapr...",['The Amity School of Engineering & Technology...,['B.Tech'],['2019'],['N/A'],[None],['Electronics'],['Coca-COla'],...,NaN,NaN,NaN,Senior Software Engineer,B.Sc in Computer Science & Engineering from a ...,At least 1 year,NaN,Technical Support\nTroubleshooting\nCollaborat...,NaN,0.850000
1,NaN,Fresher looking to join as a data analyst and ...,"['Data Analysis', 'Data Analytics', 'Business ...","['Delhi University - Hansraj College', 'Delhi ...","['B.Sc (Maths)', 'M.Sc (Science) (Statistics)']","['2015', '2018']","['N/A', 'N/A']","['N/A', 'N/A']","['Mathematics', 'Statistics']",['BIB Consultancy'],...,NaN,NaN,NaN,Machine Learning (ML) Engineer,M.Sc in Computer Science & Engineering or in a...,At least 5 year(s),NaN,Machine Learning Leadership\nCross-Functional ...,NaN,0.750000
2,NaN,NaN,"['Software Development', 'Machine Learning', '...","['Birla Institute of Technology (BIT), Ranchi']",['B.Tech'],['2018'],['N/A'],['N/A'],['Electronics/Telecommunication'],['Axis Bank Limited'],...,NaN,NaN,NaN,"Executive/ Senior Executive- Trade Marketing, ...",Master of Business Administration (MBA),At least 3 years,NaN,"Trade Marketing Executive\nBrand Visibility, S...",Brand Promotion\nCampaign Management\nField Su...,0.416667
3,NaN,To obtain a position in a fast-paced business ...,"['accounts payables', 'accounts receivables', ...","['Martinez Adult Education, Business Training ...",['Computer Applications Specialist Certificate...,['2008'],[None],[None],['Computer Applications'],"['Company Name ï¼ City , State', 'Company Name...",...,NaN,NaN,NaN,Business Development Executive,Bachelor/Honors,1 to 3 years,Age 22 to 30 years,Apparel Sourcing\nQuality Garment Sourcing\nRe...,Fast typing skill\nIELTSInternet browsing & on...,0.760000
4,NaN,Professional accountant with an outstanding wo...,"['Analytical reasoning', 'Compliance testing k...",['Kent State University'],['Bachelor of Business Administration'],[None],['3.84'],[None],['Accounting'],"['Company Name', 'Company Name', 'Company Name...",...,[None],[None],"['February 15, 2021']",Senior iOS Engineer,Bachelor of Science (BSc) in Computer Science,At least 4 years,NaN,iOS Lifecycle\nRequirement Analysis\nNative Fr...,iOS\niOS App Developer\niOS Application Develo...,0.650000


In [17]:
# for random preview
df.sample(5) 

,address,career_objective,skills,educational_institution_name,degree_names,passing_years,educational_results,result_types,major_field_of_studies,professional_company_names,...,online_links,issue_dates,expiry_dates,﻿job_position_name,educationaL_requirements,experiencere_requirement,age_requirement,responsibilities.1,skills_required,matched_score
3594,NaN,NaN,"['QA Engineering software expertise - Design, ...","['Devi Ahilya University', 'Devi Ahilya Univer...","['Master of Computer Applications (MCA)', 'Bac...","['2005', '2002']","[None, None]","[None, None]","['Computer Applications, Software Engineering,...","['Company Name', 'Company Name', 'Company Name']",...,"[None, None, None, None, None, None]","[None, None, None, None, None, None]","['May 2022', None, None, None, None, None]",Mechanical Engineer,Bachelor of Science (BSc) in Mechanical Engine...,2 to 5 years,Age 25 to 40 years,Machinery Maintenance\nTroubleshooting\nReport...,Maintenance and Troubleshooting\nMechanical,0.416667
7732,NaN,NaN,"['Microsoft Excel, Project and Visio', 'Adobe'...",NaN,NaN,NaN,NaN,NaN,NaN,"['Company Name', 'Company Name', 'Company Name...",...,NaN,NaN,NaN,Mechanical Designer,Bachelor’s degree in Mechanical Engineering fr...,At least 5 years,Age at least 24 years,Design Creation\nCAD Drawings\nDesign Optimiza...,AutoCAD\nSolidworks,0.716667
1986,NaN,Data Scientist working on problems related to ...,"['Java', 'C++', 'Python', 'Machine Learning', ...","['SGBAU University', 'Indian Institute of Tech...","['B.Tech', 'M.Tech']","['2017', '2019']","[None, None]","[None, None]","['Computer Science', 'Computer Science Enginee...",['One97 Communications Limited'],...,NaN,NaN,NaN,Data Science Engineer,Bachelor of Science (BSc) in Computer Science ...,At least 2 years,Age 25 to 32 years,Application Development\nObject-Oriented Desig...,Business Analysis\nEffective communication ski...,0.850000
4574,NaN,NaN,"['Internet', 'Microsoft Office', 'MS Word', 'M...","['University of Delaware', 'University of Dela...","['Master of Business Administration', 'Bachelo...","['2011', '2008']","['N/A', 'N/A']","[None, None]","['Finance', 'Medical Technology']","['Company Name', 'Company Name', 'Company Name']",...,NaN,NaN,NaN,DevOps Engineer,Bachelor/Honors,At least 3 years,NaN,Open-Source Technologies\nCI/CD Culture & Tool...,Ansible\nAWS Cloud\nCloud Platform\nDevOps\nDo...,0.716667
1814,NaN,NaN,"['Business to Business', 'Business to Customer...",['Grambling State University'],['Bachelor of Science'],['N/A'],['N/A'],['N/A'],['Business Management'],"['Company Name', 'Company Name', 'Company Name...",...,NaN,NaN,NaN,System Administrator (Operation & Maintenance ...,Bachelor/Honors,4 to 5 years,NaN,Hardware & Network Installation\nTroubleshooti...,Cisco\nLinux\nOperation & Maintenance of Server,0.850000


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9544 entries, 0 to 9543
Data columns (total 35 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   address                              784 non-null    object 
 1   career_objective                     4740 non-null   object 
 2   skills                               9488 non-null   object 
 3   educational_institution_name         9460 non-null   object 
 4   degree_names                         9460 non-null   object 
 5   passing_years                        9460 non-null   object 
 6   educational_results                  9460 non-null   object 
 7   result_types                         9460 non-null   object 
 8   major_field_of_studies               9460 non-null   object 
 9   professional_company_names           9460 non-null   object 
 10  company_urls                         9460 non-null   object 
 11  start_dates                   

In [19]:
missing = df.isnull().sum().sort_values(ascending=False)
missing[missing > 0]

languages                              8844
proficiency_levels                     8844
address                                8760
issue_dates                            7536
certification_skills                   7536
certification_providers                7536
expiry_dates                           7536
online_links                           7536
extra_curricular_activity_types        6118
extra_curricular_organization_links    6118
extra_curricular_organization_names    6118
role_positions                         6118
career_objective                       4804
age_requirement                        4087
skills_required                        1701
experiencere_requirement               1364
educational_results                      84
educational_institution_name             84
degree_names                             84
major_field_of_studies                   84
locations                                84
related_skils_in_job                     84
positions                       

In [20]:
# Fill missing values
for col in ["career_objective","degree_names","skills","educationaL_requirements","skills_required"]:
    df[col] = df[col].fillna("")

In [21]:
# =====================
# Text Preprocessing
# =====================
stop_words = set(stopwords.words("english"))
stemmer = PorterStemmer()

def clean_text(text):
    text = text.lower()
    text = re.sub(r"[^a-z0-9 ]", " ", text)
    words = [stemmer.stem(w) for w in text.split() if w not in stop_words]
    return " ".join(words)

df["resume_text"] = (df["career_objective"].apply(clean_text) + " " +
                     df["degree_names"].apply(clean_text) + " " +
                     df["skills"].apply(clean_text))

df["job_text"] = (df["educationaL_requirements"].apply(clean_text) + " " +
                  df["skills_required"].apply(clean_text))


In [22]:
# Remove extremely small entries
df = df[(df.resume_text.str.len()>30) & (df.job_text.str.len()>30)]

In [23]:
# =====================
# Lightweight Features
# =====================
def skill_overlap(r,j):
    r,j=set(r.split()),set(j.split())
    return len(r&j) / (len(j)+1)

df["skill_overlap"] = df.apply(lambda x:skill_overlap(x.resume_text,x.job_text),axis=1)

In [24]:
def extract_pdf(file):
    text = ""
    with pdfplumber.open(file) as pdf:
        for page in pdf.pages:
            if page.extract_text():
                text += page.extract_text() + " "
    return text

def extract_docx(file):
    doc = Document(file)
    return " ".join([p.text for p in doc.paragraphs])

def extract_txt(file):
    return file.read().decode("utf-8", errors="ignore")

def load_resume(uploaded_file):
    if uploaded_file.name.endswith(".pdf"):
        return extract_pdf(uploaded_file)
    elif uploaded_file.name.endswith(".docx"):
        return extract_docx(uploaded_file)
    elif uploaded_file.name.endswith(".txt"):
        return extract_txt(uploaded_file)
    else:
        return ""

In [25]:
# =====================
# TF-IDF (reduced size)
# =====================
vectorizer = TfidfVectorizer(max_features=6000,ngram_range=(1,2),stop_words="english")

X_text = vectorizer.fit_transform(df["resume_text"]+" "+df["job_text"])
X_numeric = csr_matrix(df[["skill_overlap"]].values)

X = hstack([X_text,X_numeric])
y = df["matched_score"]

In [26]:
# =====================
# Train-test split
# =====================
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [27]:
# =====================
# Lightweight Model
# =====================
model = Ridge(alpha=0.5)
model.fit(X_train,y_train)

y_pred=model.predict(X_test)

print("MAE:",mean_absolute_error(y_test,y_pred))
print("RMSE:",np.sqrt(mean_squared_error(y_test,y_pred)))
print("Score Example:",round(float(model.predict(X_test)[0]),2))

MAE: 0.08736689515467136
RMSE: 0.11325743447315607
Score Example: 0.85


In [28]:
# =====================
# Prediction Function
# =====================
def predict_match(resume,job):
    r,j=clean_text(resume),clean_text(job)
    vec=vectorizer.transform([r+" "+j])
    numer=csr_matrix([[skill_overlap(r,j)]])
    x=hstack([vec,numer])
    return round(model.predict(x)[0]*100,2)

print(predict_match(
    "python machine learning data analysis",
    "Looking for a machine learning engineer skilled in Python and data analysis."
))

76.57


In [29]:
pickle.dump(model, open("model.pkl", "wb"))
pickle.dump(vectorizer, open("vectorizer.pkl", "wb"))